## Imports


In [ ]:
import ee
import matplotlib.pyplot as plt
import pandas as pd

## GEE authentication

In [ ]:
ee.Authenticate()
ee.Initialize()

Test if authentication was succesful

In [ ]:
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## LEAF-toolbox imports

Modules imports cannot be done before GEE authentication have been run
succesfuly

In [ ]:
from leaftoolbox import LEAF
from leaftoolbox import SL2PV0 
from leaftoolbox import SL2PV1

# Process Collection of Input Images Returning GEE Collection of Output Products

In [ ]:
imageCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                    .filterDate('2020-08-01', '2020-08-16') \
                    .filter(ee.Filter.eq('MGRS_TILE','18TVQ'))
print(imageCollection.size().getInfo())

products = LEAF.makeProductImageCollection(imageCollection=imageCollection,algorithm=SL2PV0,variable='LAI',inputScaleSize=40) 
print(products.first().bandNames().getInfo())

# Process Sites with output as a Pandas Data Frame

In [ ]:
siteList = ["projects/ee-modis250/assets/HuntClub",
            "projects/ee-modis250/assets/Nanaimo",
			"projects/ee-modis250/assets/Karachi"]

# Sample Surface Reflectance from various sensors over sites

In [ ]:
#Landsat 8
sitesDictionaryL08SR = LEAF.sampleSites(siteList, \
                                                imageCollectionName='LANDSAT/LC08/C02/T1_L2',\
                                                algorithm=SL2PV0, 
                                                variableName="Surface_Reflectance",\
                                                maxCloudcover=90,\
                                                outputScaleSize=30,\
                                                inputScaleSize=30,\
                                                bufferSpatialSize = 0,\
                                                bufferTemporalSize =["2021-04-01","2022-10-01"],  \
                                                subsamplingFraction = 0.99) 
# Landsat 9
sitesDictionaryL09SR = LEAF.sampleSites(siteList, \
                                                imageCollectionName='LANDSAT/LC09/C02/T1_L2',\
                                                algorithm=SL2PV0, 
                                                variableName="Surface_Reflectance",\
                                                outputScaleSize=30,\
                                                inputScaleSize=30,\
                                                bufferSpatialSize = 0,\
                                                bufferTemporalSize =["2021-04-01","2022-10-01"],  \
                                                subsamplingFraction = 0.99)

In [ ]:
 #Landsat 8
sitesDictionaryL08V0 = LEAF.sampleSites(siteList, \
                                                 imageCollectionName='LANDSAT/LC08/C02/T1_L2',\
                                                 algorithm=SL2PV0, 
                                                 variableName="LAI",\
                                                 maxCloudcover=90,\
                                                 outputScaleSize=30,\
                                                 inputScaleSize=30,\
                                                 bufferSpatialSize = 0,\
                                                 bufferTemporalSize =["2021-04-01","2022-10-01"],  \
                                                 subsamplingFraction = 0.99) 
# Landsat 9
sitesDictionaryL09V0 = LEAF.sampleSites(siteList, \
                                                imageCollectionName='LANDSAT/LC09/C02/T1_L2',\
                                                algorithm=SL2PV0, 
                                                variableName="LAI",\
                                                maxCloudcover=90,\
                                                outputScaleSize=30,\
                                                inputScaleSize=30,\
                                                bufferSpatialSize = 0,\
                                                bufferTemporalSize =["2021-04-01","2022-10-01"],  \
                                                subsamplingFraction = 0.99)
#HLS
sitesDictionaryHLSV0 = LEAF.sampleSites(siteList, \
                                                imageCollectionName="NASA/HLS/HLSL30/v002",\
                                                algorithm=SL2PV0, 
                                                variableName="LAI",\
                                                maxCloudcover=90,\
                                                outputScaleSize=30,\
                                                inputScaleSize=30,\
                                                bufferSpatialSize = 0,\
                                                bufferTemporalSize =["2021-04-01","2022-10-01"],  \
                                                subsamplingFraction = 0.99)         

# S2
sitesDictionaryS2V0 = LEAF.sampleSites(siteList, \
                                                imageCollectionName="COPERNICUS/S2_SR_HARMONIZED",\
                                                algorithm=SL2PV0, 
                                                variableName="LAI",\
                                                maxCloudcover=90,\
                                                outputScaleSize=20,\
                                                inputScaleSize=20,\
                                                bufferSpatialSize = 0,\
                                                bufferTemporalSize =["2021-04-01","2022-10-01"],  \
                                                subsamplingFraction = 0.99)

# Visualize Results


In [ ]:
# Choose a site from the sitelist 
siteNum=0

# Select the first feature 
featureNum = 0

#Select one sampled pixel from each feature 
pixelNum = 3

#Extract time series of LAI with high quality only
site = sitesDictionaryL08V0[siteList[siteNum]]
# print(site[featureNum]['leaftoolbox.SL2PV0'])
df=site[featureNum]['leaftoolbox.SL2PV0']
df['utc'] =  pd.to_datetime(df['date'],unit='ms')
pixelL08V0=df.loc[(df['longitude']==df.loc[pixelNum].longitude) & (df['latitude']==df.loc[pixelNum].latitude) & (df['QC']==0)]
site=sitesDictionaryL09V0[siteList[siteNum]]
df=site[featureNum]['leaftoolbox.SL2PV0']
df['utc'] =  pd.to_datetime(df['date'],unit='ms')
pixelL09V0=df.loc[(df['longitude']==df.loc[pixelNum].longitude) & (df['latitude']==df.loc[pixelNum].latitude) & df['QC']==0]
site=sitesDictionaryHLSV0[siteList[siteNum]]
df=site[featureNum]['leaftoolbox.SL2PV0']
df['utc'] =  pd.to_datetime(df['date'],unit='ms')
pixelHLSV0=df.loc[(df['longitude']==df.loc[pixelNum].longitude) & (df['latitude']==df.loc[pixelNum].latitude) & df['QC']==0]
site=sitesDictionaryS2V0[siteList[siteNum]]
df=site[featureNum]['leaftoolbox.SL2PV0']
df['utc'] =  pd.to_datetime(df['date'],unit='ms')
pixelS2V0=df.loc[(df['longitude']==df.loc[pixelNum].longitude) & (df['latitude']==df.loc[pixelNum].latitude) & df['QC']==0]

fig,ax = plt.subplots(1,1,figsize=[10,10])
plt.plot(pixelL08V0['utc'],pixelL08V0['estimateLAI'],'ob',markerfacecolor='none', label='L08V1')
plt.plot(pixelL09V0['utc'],pixelL09V0['estimateLAI'],'ok',markerfacecolor='none', label='L09V1')
plt.plot(pixelHLSV0['utc'],pixelHLSV0['estimateLAI'],'oc',markerfacecolor='none', label='HLSV1')
plt.plot(pixelS2V0['utc'],pixelS2V0['estimateLAI'],'or',markerfacecolor='none',label='S2V1')

ax.legend()
ax.set_xlabel('date')
ax.set_ylabel('LAI')
plt.xticks(rotation=90);